In [ ]:
import uuid

import networkx as nx
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
setup = {
    'drop' : [0,1],
    'personal' : [0,1],
    'veterano' : [3],
}

topic_to_idx = {
    'multiple_choice' : [2],
    'binary' : [3,4],
    'numerical' : [6],
    'embedding' : [5,7,8,9],
}

In [209]:
total_df = pd.read_csv('../test.csv')

total_df.columns = total_df.columns.str.replace(r'[^\w]', '_', regex=True)

total_df['ID'] = [str(uuid.uuid4()) for _ in range(len(total_df))]
total_df = total_df.drop(total_df.columns[setup['drop']], axis=1)

personal_df = total_df.iloc[:, setup['personal']+setup['veterano']].copy()

personal_df['ID'] = total_df['ID']

df = total_df.drop(total_df.columns[setup['personal']], axis=1).sample(frac=1).reset_index(drop=True)

In [291]:
total_df

,Email,Nome,Curso,Você_é_veterane__turma_anterior_a_024__ou_bixe__024_,Pronomes,Etnia,Você_é_membro_da_comunidade_LGBTQIAP__,Você_prefere_role_ou_é_mais_caseiro,Qual_a_sua_idade__apenas_numero_,de_qual_cidade_estado_voce_é_,Quais_hobbies_vc_tem__esportes__jogos__,qual_estilo_de_musica_preferido,ID
0,marcos100@example.com,Marcos Oliveira,CC,Bixe,Ele/Dele,Branco,Sim,role,18,São Paulo/SP,"Futebol, leitura","Sertanejo, pop",8f3f55ac-669f-4388-b1df-8a23b573eacf
1,ana101@example.com,Ana Souza,EC,Veterane,Ela/Dela,Indígena,Sim,caseiro,22,Cuiabá/MT,"Yoga, leitura","MPB, reggae",d3877ac5-c88a-436f-bae4-20a03217222e
2,ricardo102@example.com,Ricardo Costa,CC,Bixe,Ele/Dele,Negra,Sim,role,19,Rio de Janeiro/RJ,"Futebol, natação","Rap, samba",93c655e2-07a1-4717-965e-50dc88ceb46b
3,luiza103@example.com,Luiza Lima,EC,Veterane,Ela/Dela,Branca,Não,caseiro,23,Manaus/AM,"Leitura, culinária","Jazz, MPB",d0afb949-cfe6-4bf8-bedb-db7f35cd4db7
4,fernando104@example.com,Fernando Pereira,CC,Bixe,Ele/Dele,Pardo,Sim,role,18,São Luís/MA,"Jogos, cinema","Rock, indie",d474ea75-72c4-48b9-bf94-7d7fb71511e7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,andré191@example.com,André Ribeiro,CC,Bixe,Ele/Dele,Pardo,Sim,role,19,Curitiba/PR,"Esportes, fotografia","Reggae, jazz",442ca75c-d2e9-4967-8893-679d7b7d8d57
92,luiz192@example.com,Luiz Santos,EC,Veterane,Ele/Dele,Branco,Não,caseiro,22,Recife/PE,"Yoga, culinária","Pop, samba",818eab7f-b9c2-44db-a750-a06be60ec0ba
93,lucas193@example.com,Lucas Rocha,CC,Bixe,Ele/Dele,Pardo,Sim,role,20,Fortaleza/CE,"Jogos, esportes","MPB, indie",e164a039-5258-47f0-b2d6-1e1a5eebb6ee
94,rodrigo194@example.com,Rodrigo Almeida,CC,Bixe,Ele/Dele,Branco,Sim,role,18,Florianópolis/SC,"Corrida, dança","Rock, reggae",3cf658b3-dce5-4506-803f-39566993496d


In [210]:
df.head()

,Curso,Você_é_veterane__turma_anterior_a_024__ou_bixe__024_,Pronomes,Etnia,Você_é_membro_da_comunidade_LGBTQIAP__,Você_prefere_role_ou_é_mais_caseiro,Qual_a_sua_idade__apenas_numero_,de_qual_cidade_estado_voce_é_,Quais_hobbies_vc_tem__esportes__jogos__,qual_estilo_de_musica_preferido,ID
0,CC,Bixe,Ele/Dele,Branco,Sim,role,19,São Paulo/SP,"Jogos, leitura","Rock, MPB",b2523adf-f3ee-4bbe-82b8-fed3601bf3e2
1,EC,Veterane,Ela/Dela,Branca,Não,caseiro,22,Rio de Janeiro/RJ,"Yoga, dança","Jazz, indie",88dce958-278a-478b-973a-9b01f4781195
2,EC,Veterane,Ela/Dela,Indígena,Sim,caseiro,23,Florianópolis/SC,"Culinária, leitura","Jazz, indie",21ae2d7f-3fb5-4ea2-b159-b50352694d36
3,EC,Veterane,Ela/Dela,Preto,Não,caseiro,24,Porto Alegre/RS,"Fotografia, dança","Samba, pop",57ba7a5b-7a50-44f6-92b6-8a9dcf1c2736
4,CC,Bixe,Ele/Dele,Negro,Sim,role,20,Salvador/BA,"Futebol, leitura","Reggae, pop",78054aec-9a18-4912-9e0e-628ca285fb55


In [211]:
personal_df.head()

,Email,Nome,Você_é_veterane__turma_anterior_a_024__ou_bixe__024_,ID
0,marcos100@example.com,Marcos Oliveira,Bixe,8f3f55ac-669f-4388-b1df-8a23b573eacf
1,ana101@example.com,Ana Souza,Veterane,d3877ac5-c88a-436f-bae4-20a03217222e
2,ricardo102@example.com,Ricardo Costa,Bixe,93c655e2-07a1-4717-965e-50dc88ceb46b
3,luiza103@example.com,Luiza Lima,Veterane,d0afb949-cfe6-4bf8-bedb-db7f35cd4db7
4,fernando104@example.com,Fernando Pereira,Bixe,d474ea75-72c4-48b9-bf94-7d7fb71511e7


In [212]:
padrinho_col = df.columns[1]
curso_col = df.columns[0]
nome_col = personal_df.columns[1]

In [213]:
topic_to_cols = {
    topic : [df.columns[i] for i in idx] 
    for topic, idx in topic_to_idx.items()
}

In [214]:
topic_to_cols   

{'multiple_choice': ['Pronomes'],
 'binary': ['Etnia', 'Você_é_membro_da_comunidade_LGBTQIAP__'],
 'numerical': ['Qual_a_sua_idade__apenas_numero_'],
 'embedding': ['Você_prefere_role_ou_é_mais_caseiro',
  'de_qual_cidade_estado_voce_é_',
  'Quais_hobbies_vc_tem__esportes__jogos__',
  'qual_estilo_de_musica_preferido']}

In [373]:
topic_to_weights = {
    'multiple_choice' : [8],
    'binary' : [2,1],
    'numerical' : [1],
    'embedding' : [1,1,1,1],
}

In [374]:
par_idx = df[df[padrinho_col] == 'Veterane'].index.to_list()
chi_idx = df.index.difference(par_idx).to_list()

In [375]:
def apply_restrictions():
    pass

In [ ]:
def normalize_func(row):
    # exp_values = np.exp(row - np.max(row))  
    # return exp_values / np.sum(exp_values)

    return row / np.sum(row)
    # mais contrastanete pra valores binarios, melhora na aplicacao do peso

In [377]:
def normalize_rows(mat):
    normalized_mat = np.apply_along_axis(
        normalize_func, axis=1, arr=mat
    )

    return normalized_mat

In [378]:
def handle_topic_weights(topic, weight_func):
    col_to_weights = {}
    for col in topic_to_cols[topic]:
        
        par_resp = df.loc[par_idx, col].to_numpy()
        chi_resp = df.loc[chi_idx, col].to_numpy()

        weight_mat = weight_func(chi_resp, par_resp)

        apply_restrictions()

        col_to_weights[col] = normalize_rows(weight_mat)

    return col_to_weights


In [379]:
embedder = SentenceTransformer('all-mpnet-base-v2')

In [380]:
def get_embed_weight(chi_resp, par_resp):
    par_embed = embedder.encode(par_resp)   
    chi_embed = embedder.encode(chi_resp)

    weight_mat = cosine_similarity(chi_embed, par_embed)

    return weight_mat

In [381]:
def num_func(x1, x2):
    return 1 - (x1 - x2)**2

In [382]:
def get_num_weight(chi_resp, par_resp):
    par_numeric = np.array([int(x) if str(x).isnumeric() else 18 for x in par_resp])
    chi_numeric = np.array([int(x) if str(x).isnumeric() else 18 for x in chi_resp])

    combined = np.concatenate((par_numeric, chi_numeric))
    diff = combined.max() - combined.min()
    
    par_numeric = par_numeric / diff
    chi_numeric = chi_numeric / diff

    weight_mat = np.zeros((len(chi_numeric), len(par_numeric)))

    for i, x1 in enumerate(chi_numeric):
        weight_mat[i, :] = np.vectorize(num_func)(x1, par_numeric)

    return weight_mat


In [383]:
def get_binary_weight(chi_resp, par_resp):

    weight_mat = np.zeros((len(chi_resp), len(par_resp)))

    for i, chi in enumerate(chi_resp):
        for j, par in enumerate(par_resp):
            weight_mat[i, j] = chi == par

    return weight_mat

In [384]:
def get_multiple_choice_weight(chi_resp, par_resp):

    weight_mat = np.zeros((len(chi_resp), len(par_resp)))

    par_resp = [x.split(';') for x in par_resp] 
    chi_resp = [x.split(';') for x in chi_resp]
    
    for i, chi in enumerate(chi_resp):
        for j, par in enumerate(par_resp):
            weight_mat[i, j] = any(element in chi for element in par)

    return weight_mat

In [385]:
col_to_weights = {
    **handle_topic_weights('multiple_choice', get_multiple_choice_weight),
    **handle_topic_weights('binary', get_binary_weight),
    **handle_topic_weights('numerical', get_num_weight),
    **handle_topic_weights('embedding', get_embed_weight),
}

/tmp/ipykernel_137511/2946488421.py:5: RuntimeWarning: invalid value encountered in divide
  return row / np.sum(row)


In [386]:
col_to_weights

{'Pronomes': array([[0.        , 0.        , 0.        , ..., 0.        , 0.07142857,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.07142857,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.07142857,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.07142857,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.07142857,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.07142857,
         0.        ]]),
 'Etnia': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'Você_é_membro_da_comunidade_LGBTQIAP__': array([[0.        , 0.16666667, 0.        , ..., 0.        , 0.        ,
         0.        ],
 

In [387]:
edges = np.zeros((len(chi_idx), len(par_idx)))

for topic, cols in topic_to_cols.items():
    for i, col in enumerate(cols):
        edges += col_to_weights[col] * topic_to_weights[topic][i]

weight_sum = sum(np.sum(weights) for weights in topic_to_weights.values())

edges = edges / weight_sum

In [388]:
edges

array([[0.00852603, 0.0193437 , 0.00744263, ..., 0.00978223, 0.04462401,
        0.00863195],
       [0.00848763, 0.0196628 , 0.00794936, ..., 0.01223912, 0.04538636,
        0.00813012],
       [0.0083507 , 0.01928677, 0.00823585, ..., 0.01044657, 0.04663168,
        0.00838714],
       ...,
       [0.00866761, 0.01801154, 0.01132236, ..., 0.00880969, 0.04688958,
        0.00914767],
       [0.01081097, 0.02129202, 0.00738224, ..., 0.00891026, 0.04549507,
        0.00956444],
       [0.00865239, 0.01864105, 0.00876563, ..., 0.00904165, 0.04529415,
        0.00829764]])

In [389]:
par_to_mat = { idx : i for i, idx in enumerate(par_idx) }
chi_to_mat = { idx : i for i, idx in enumerate(chi_idx) }

In [390]:
def distribute_random(n, max_n, max_chi_par):
    distribution = np.ones(n, dtype=int) 
    diff = max_n - n

    available_indices = set(range(n))

    while diff > 0:
        idx = np.random.choice(list(available_indices))
        
        distribution[idx] += 1
        diff -= 1
        
        if distribution[idx] == max_chi_par:
            available_indices.remove(idx)

    return distribution


In [391]:
n_chi = len(chi_idx)
n_par = len(par_idx)

max_chi_par = 2

diff = n_chi - n_par

max_matches = min(n_chi, n_par)*max_chi_par

chi_dist = distribute_random(n_chi, max_matches, max_chi_par)
par_dist = distribute_random(n_par, max_matches, max_chi_par)

chi_ids = np.array([f'{str(chi_idx[i])}_{j}' for i in range(len(chi_idx)) for j in range(chi_dist[i])])
par_ids = np.array([f'{str(par_idx[i])}_{j}' for i in range(len(par_idx)) for j in range(par_dist[i])])
# duplica os pais e filhos pra terem o mesmo numero 

G = nx.Graph()
G.add_nodes_from(chi_ids, bipartite=0)
G.add_nodes_from(par_ids, bipartite=1)

for chi in chi_ids:
    for par in par_ids:
        i = chi_to_mat[int(chi.split('_')[0])]
        j = par_to_mat[int(par.split('_')[0])]

        weight = edges[i, j]
        if int(chi.split('_')[1]) > 0 and int(par.split('_')[0]) > 0:
            weight = 0
        # de for repetido tanto o filho quanto o pai, peso eh zero pra nao repetir 
        # condicao pode ser amenizada pra melhorar o algoritmo em si, mas e ajustavel com os pesos dos topicos

        G.add_edge(chi, par, weight=weight)
        
        # precisa usar id do dataframe, porque matching nao necessariamente sai num formato fixo
        # nao da pra usar relativo à matriz

matching = nx.matching.max_weight_matching(G, maxcardinality=True)

print(matching)



{(np.str_('2_1'), np.str_('9_0')), (np.str_('42_0'), np.str_('70_1')), (np.str_('33_0'), np.str_('27_1')), (np.str_('78_0'), np.str_('22_0')), (np.str_('2_0'), np.str_('63_0')), (np.str_('64_0'), np.str_('20_0')), (np.str_('12_1'), np.str_('93_0')), (np.str_('24_0'), np.str_('66_1')), (np.str_('83_0'), np.str_('15_0')), (np.str_('70_0'), np.str_('58_0')), (np.str_('34_0'), np.str_('61_1')), (np.str_('59_0'), np.str_('74_1')), (np.str_('39_0'), np.str_('52_0')), (np.str_('16_0'), np.str_('77_0')), (np.str_('10_0'), np.str_('73_1')), (np.str_('76_0'), np.str_('49_0')), (np.str_('28_0'), np.str_('5_0')), (np.str_('79_1'), np.str_('26_1')), (np.str_('71_0'), np.str_('35_0')), (np.str_('79_0'), np.str_('33_1')), (np.str_('1_0'), np.str_('13_0')), (np.str_('38_0'), np.str_('55_1')), (np.str_('61_0'), np.str_('39_1')), (np.str_('26_0'), np.str_('43_1')), (np.str_('74_0'), np.str_('21_0')), (np.str_('54_0'), np.str_('3_1')), (np.str_('55_0'), np.str_('40_0')), (np.str_('89_0'), np.str_('90_1')

In [392]:
uid_to_row = { row['ID'] : i for i, row in personal_df.iterrows() }
uid_to_row

{'8f3f55ac-669f-4388-b1df-8a23b573eacf': 0,
 'd3877ac5-c88a-436f-bae4-20a03217222e': 1,
 '93c655e2-07a1-4717-965e-50dc88ceb46b': 2,
 'd0afb949-cfe6-4bf8-bedb-db7f35cd4db7': 3,
 'd474ea75-72c4-48b9-bf94-7d7fb71511e7': 4,
 '57ba7a5b-7a50-44f6-92b6-8a9dcf1c2736': 5,
 'f31e7385-4123-428f-9011-c0eae2fe3520': 6,
 'b488d076-f186-46bf-a86e-348f25b3b330': 7,
 '72da4507-6d64-42be-8ce3-6ea584ba25af': 8,
 '7faf91aa-810d-48d3-8bfb-54f7b5aac921': 9,
 '0fb105d9-cc94-42ad-b54a-cbc1d2d6f94e': 10,
 '45099cdc-18e2-448b-89f9-f8767276a861': 11,
 'f70aeb7b-6276-4b41-8588-b2f44d1f063f': 12,
 '6b79ef0e-f5ca-41dd-9eeb-41471e294f2a': 13,
 '9889a2af-79a3-46ae-8de8-d3d33b818a5f': 14,
 '534f7aa9-b86a-4b6e-b67f-7c513ee36d63': 15,
 'a94f42db-f2d4-49d0-9b3f-affd60159c85': 16,
 '38187a7e-a186-4776-8100-9b0d464ce9ca': 17,
 '433a1ce6-6196-495b-92fe-3097a3726b74': 18,
 '73592397-6260-41a2-b348-6214c2917398': 19,
 'cf83843e-776e-4754-ab04-82472ac647c0': 20,
 'c4d497f7-7751-4191-9679-371ed097321a': 21,
 '21ae2d7f-3fb5-4ea2

In [393]:
padrinho_nomes = personal_df[personal_df[padrinho_col] == 'Veterane'][nome_col].to_list()
bixos_nomes = personal_df[personal_df[padrinho_col] == 'Bixe'][nome_col].to_list()

padrinho_to_bixo = { nome : [] for nome in padrinho_nomes }
bixo_to_padrinho = { nome : [] for nome in bixos_nomes }

for p1_idx, p2_idx in matching:
    p1_personal_row = uid_to_row[df.loc[int(p1_idx.split('_')[0]), 'ID']]
    p2_personal_row = uid_to_row[df.loc[int(p2_idx.split('_')[0]), 'ID']]

    p1_nome = personal_df.loc[p1_personal_row, nome_col]
    p2_nome = personal_df.loc[p2_personal_row, nome_col]

    if personal_df.loc[p1_personal_row, padrinho_col] == "Veterane":
        padrinho_to_bixo[p1_nome].append(p2_nome)
        bixo_to_padrinho[p2_nome].append(p1_nome)
    else:
        padrinho_to_bixo[p2_nome].append(p1_nome)
        bixo_to_padrinho[p1_nome].append(p2_nome)


print(padrinho_to_bixo)
print(bixo_to_padrinho)


{'Ana Souza': ['Gustavo Almeida', 'Felipe Lima'], 'Luiza Lima': ['Marcos Lima', 'Gustavo Almeida'], 'Mariana Silva': ['Marcos Lima', 'Rodrigo Santos', 'Vinícius Mendes', 'Bruno Silva'], 'Roberta Andrade': ['Bruna Pereira', 'Rodrigo Almeida'], 'Patrícia Costa': ['Vinícius Mendes', 'Marcos Oliveira'], 'Renata Silva': ['Pedro Lima', 'Ana Silva'], 'Carlos Oliveira': ['Lucas Santos', 'Gustavo Lima'], 'João Almeida': ['Mateus Rocha', 'Bruno Nogueira', 'Lucas Ribeiro', 'Paulo Souza', 'Marcos Cardoso', 'Gustavo Pereira'], 'Mariana Castro': ['Bruno Ferreira', 'Carlos Pereira'], 'Luiza Costa': ['Tiago Mendes', 'Carlos Souza'], 'Jessica Oliveira': ['Luiz Mendes', 'Rafael Almeida'], 'Roberta Souza': ['Felipe Souza', 'Tiago Mendes'], 'Fernanda Costa': ['Daniel Nunes', 'Eduardo Rocha'], 'Marcos Souza': ['Carlos Nunes', 'Paulo Nogueira'], 'Patrícia Lima': ['Andréa Martins', 'Matheus Almeida'], 'Aline Cardoso': ['Rafael Souza', 'Carlos Andrade'], 'Juliana Nunes': ['Lucas Ribeiro', 'Felipe Nogueira'], 